<a href="https://colab.research.google.com/github/g-roger/plataformas-cognitivas/blob/main/Plataformas_Azure_Treino_Fora_Azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
!git clone https://github.com/elthonf/azure-ml.git

Cloning into 'azure-ml'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 105 (delta 41), reused 83 (delta 24), pack-reused 0
Receiving objects: 100% (105/105), 6.01 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [2]:
!pip install azureml-core
!pip install scikit-learn==0.20.3

     |████████████████████████████████| 2.4 MB 5.0 MB/s 
     |████████████████████████████████| 928 kB 25.0 MB/s 
     |████████████████████████████████| 547 kB 13.6 MB/s 
     |████████████████████████████████| 1.3 MB 39.3 MB/s 
     |████████████████████████████████| 40 kB 4.6 MB/s 
     |████████████████████████████████| 146 kB 40.4 MB/s 
     |████████████████████████████████| 3.0 MB 35.1 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 412 kB 47.6 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 54 kB 2.0 MB/s 
     |████████████████████████████████| 141 kB 48.7 MB/s 
     |████████████████████████████████| 178 kB 60.3 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
     |████████████████████████████████| 54 kB 1.9 MB/s 
     |█████████████████████████

In [3]:
!pip install azureml-dataprep
!pip install azureml-dataset-runtime

     |████████████████████████████████| 39.4 MB 1.2 MB/s 
     |████████████████████████████████| 29.3 MB 1.5 MB/s 
     |████████████████████████████████| 1.3 MB 54.5 MB/s 
     |████████████████████████████████| 13.2 MB 49.7 MB/s 
     |████████████████████████████████| 129 kB 52.3 MB/s 
     |████████████████████████████████| 78 kB 5.4 MB/s 
     |████████████████████████████████| 39.4 MB 1.3 MB/s 
     |████████████████████████████████| 13.1 MB 36.6 MB/s 
  Attempting uninstall: azureml-dataprep-rslex
    Found existing installation: azureml-dataprep-rslex 2.1.0
    Uninstalling azureml-dataprep-rslex-2.1.0:
      Successfully uninstalled azureml-dataprep-rslex-2.1.0
  Attempting uninstall: azureml-dataprep
    Found existing installation: azureml-dataprep 2.25.0
    Uninstalling azureml-dataprep-2.25.0:
      Successfully uninstalled azureml-dataprep-2.25.0


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
import sklearn
import math
from azureml.core import Experiment
import random
import pandas as pd
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import RandomForestRegressor as rfr
import uuid
# O azureml-core da versão 1.0.72 ou superior é requerido
# é necessário azureml-dataprep[pandas] na versão 1.1.34 ou superior
from azureml.core import Workspace, Dataset

In [79]:
from azureml.core import Model
import azureml

In [80]:
azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = 1000000000

In [6]:
def get_a_funnyName():
    f = open("azure-ml/datasets/names/funnynames.txt", "r")
    nomes = f.readlines()
    return random.choice(nomes).rstrip('\n')

# Conexão

In [7]:
#Trocar os códigos abaixo pelos da sua instância!
subscription_id = 'de1900a3-409d-4d98-acfd-b2011279ed90'
resource_group = 'groger-ia'
workspace_name = 'Workspace20ia'

workspace = Workspace(subscription_id, resource_group, workspace_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RB724RLWR to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


# Carga de dados

In [8]:
mydf = pd.read_csv('azure-ml/datasets/statistical/BaseDefault01.csv')
mydf.head()

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade,default
0,"Simon, Rodriguez",4472.190323,42.036031,0,0,1,0,0,3,0
1,"Daniel, Castro",4592.774312,48.230662,1,0,1,0,1,2,0
2,"Myhue, Lin",2486.538807,56.881709,0,1,0,0,0,0,1
3,"Destiny, Richardson-Pacheco",2852.340117,51.684021,1,1,0,0,0,2,1
4,"Brittany, Cohen-Wilson",4703.782812,50.729078,1,1,1,0,1,2,0


In [9]:
dataset = Dataset.get_by_name(workspace, name='Risco-De-Credito')
df = dataset.to_pandas_dataframe()
df.head()

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade,default
0,"Simon, Rodriguez",4472.190323,42.036031,0,0,1,0,0,3,0
1,"Daniel, Castro",4592.774312,48.230662,1,0,1,0,1,2,0
2,"Myhue, Lin",2486.538807,56.881709,0,1,0,0,0,0,1
3,"Destiny, Richardson-Pacheco",2852.340117,51.684021,1,1,0,0,0,2,1
4,"Brittany, Cohen-Wilson",4703.782812,50.729078,1,1,1,0,1,2,0


# Treinamento

Identifica no dataset as variáveis independentes e a variavel alvo

In [10]:
targetcol = 'default'
y = mydf[targetcol]

Cria experimento

In [71]:
experiment = Experiment(workspace=workspace, name="Experimento_Groger")

## Treina modelo 01 (Classificador)

Cria uma execução para o experimento

In [81]:
run = experiment.start_logging(run_id= "Classificador1_" + str( uuid.uuid1() ) )
run.log("Tipo", "Classificador")

In [82]:
independentcols = ['renda', 'idade', 'etnia', 'sexo', 'casapropria', 'outrasrendas', 'estadocivil', 'escolaridade']
x = mydf[independentcols]
clf = rfc()
clf.fit(X=x, y=y)
clf.independentcols = independentcols

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Avaliações e Logs

In [83]:
clf_acuracia = clf.score(X=x, y=y)
print("Modelo 01 (classificador), criado com acurácia de: [{0}]".format(clf_acuracia))
run.log("acuracia", clf_acuracia)

Modelo 01 (classificador), criado com acurácia de: [0.98745]


In [84]:
#Demais logs (não serão comparados)
run.log("Versao sklearn", sklearn.__version__)
run.log("criterion", clf.criterion)
run.log("n_estimators", clf.n_estimators)
run.log("min_samples_leaf", clf.min_samples_leaf)
run.log("max_depth",  str(clf.max_depth))
run.log_list("Inputs", independentcols)

Exporta o modelo

In [85]:
model_name = "nome_arquivo.pkl"
filename = "outputs/" + model_name
joblib.dump(value=clf, filename=filename)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

['outputs/nome_arquivo.pkl']

Upload do arquivo, deve demorar um pouco

In [86]:
run.upload_file(name=model_name, path_or_stream=filename)

In [87]:
ls -l outputs

total 299324
-rw-r--r-- 1 root root 58056305 Dec  8 01:45 grid-fit-clf.pkl
-rw-r--r-- 1 root root 58056305 Dec  8 01:54 grid-random.pkl
-rw-r--r-- 1 root root 58056305 Dec  8 01:54 grid-random-x.pkl
-rw-r--r-- 1 root root 58056305 Dec  8 01:46 nome_arquivo2.pkl
-rw-r--r-- 1 root root 16220191 Dec  8 02:24 nome_arquivo.pkl
-rw-r--r-- 1 root root 58056305 Dec  8 01:57 nome_arquivoZ.pkl


Finaliza o experimento

In [88]:
run.complete()

# Atividade 1 : Análise na Azure

Crie n modelos, e envie para a Azure. Em seguida, compare os resultados dentro da plataforma.  

Obs.:  
n = 8 $\times$ 4 $\times$ 3

In [65]:
n_estimators = [1, 2, 3, 5, 10, 20, 30, 100]
max_depth = [1, 2, 3, None]
min_samples_leaf = [10, 5, 1]


# parametros = {
#   'n_estimators': [1, 2, 3, 5, 10, 20, 30, 100],
#   'max_depth': [1, 2, 3, None],
#   'min_samples_leaf': [10, 5, 1]
# }

In [21]:
# clf = rfc()

In [40]:
# from sklearn.metrics import make_scorer

# # Importando os módulos de cálculo de métricas
# from sklearn.metrics import precision_score
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import fbeta_score

# # Criando um dicionário com as métricas que desejo calcular.
# meus_scores = {'accuracy' :make_scorer(accuracy_score),
#                'recall'   :make_scorer(recall_score),
#                'precision':make_scorer(precision_score),
#                'f1'       :make_scorer(fbeta_score, beta = 1)}

# # Exemplo para o uso scoring igual ao meus_scores.
# grid = GridSearchCV(estimator = clf,
#                       param_grid = parametros,
#                       cv = 5,
#                       scoring = meus_scores,   # É o meus_scores
#                       refit = 'f1') 

In [41]:
# grid.fit(X=x, y=y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [1, 2, 3, 5, 10, 20, 30, 100], 'max_depth': [1, 2, 3, None], 'min_samples_leaf': [10, 5, 1]},
       pre_dispatch='2*n_jobs', refit='f1', return_train_score='warn',
       scoring={'accuracy': make_scorer(accuracy_score), 'recall': make_scorer(recall_score), 'precision': make_scorer(precision_score), 'f1': make_scorer(fbeta_score, beta=1)},
       verbose=0)

In [51]:
# run.log("score", meus_scores)

In [62]:
# model= "nome_arquivoZ.pkl"
# filename = "outputs/" + model
# joblib.dump(value=grid, filename=filename)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

['outputs/nome_arquivoZ.pkl']

In [89]:
for estimators in n_estimators:
  for depths in max_depth:
    for leafs in min_samples_leaf:
      run = experiment.start_logging(run_id="Classificador1_" + str( uuid.uuid1() ) )
      run.log("Tipo", "Classificador")

      independentcols = ['renda', 'idade', 'etnia', 'sexo', 'casapropria', 'outrasrendas', 'estadocivil', 'escolaridade']
      x = mydf[independentcols]
      clf = rfc(n_estimators=estimators, max_depth=depths, min_samples_leaf=leafs)
      clf.fit(X=x, y=y)
      clf.independentcols = independentcols
      clf_acuracia = clf.score(X=x, y=y)
      print("Modelo 01 (classificador), criado com acurácia de: [{0}]".format(clf_acuracia))
      run.log("acuracia", clf_acuracia)

      run.log("Versao sklearn", sklearn.__version__)
      run.log("criterion", clf.criterion)
      run.log("n_estimators", clf.n_estimators)
      run.log("min_samples_leaf", clf.min_samples_leaf)
      run.log("max_depth",  str(clf.max_depth))
      run.log_list("Inputs", independentcols)

      model_name = "nome_arquivo.pkl"
      filename = "outputs/" + model_name
      joblib.dump(value=clf, filename=filename)

      run.upload_file(name=model_name, path_or_stream=filename)

      run.complete()

Modelo 01 (classificador), criado com acurácia de: [0.63969]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

Modelo 01 (classificador), criado com acurácia de: [0.7429]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

Modelo 01 (classificador), criado com acurácia de: [0.7429]


...

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

Modelo 01 (classificador), criado com acurácia de: [0.91455]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

Modelo 01 (classificador), criado com acurácia de: [0.99995]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

# Atividade 2 : Regressor

Treinar um segundo modelo (Regressor sem a variavel etnia)

In [90]:
for estimators in n_estimators:
  for depths in max_depth:
    for leafs in min_samples_leaf:
      run = experiment.start_logging(run_id="Regressor_1" + str( uuid.uuid1() ) )
      run.log("Tipo", "Regressor")

      independentcols = ['renda', 'idade', 'sexo', 'casapropria', 'outrasrendas', 'estadocivil', 'escolaridade']
      x = mydf[independentcols]
      clf = rfr(n_estimators=estimators, max_depth=depths, min_samples_leaf=leafs)
      clf.fit(X=x, y=y)
      clf.independentcols = independentcols
      clf_acuracia = clf.score(X=x, y=y)
      print("Modelo 02 (Regressor), criado com acurácia de: [{0}]".format(clf_acuracia))
      run.log("acuracia", clf_acuracia)

      run.log("Versao sklearn", sklearn.__version__)
      run.log("criterion", clf.criterion)
      run.log("n_estimators", clf.n_estimators)
      run.log("min_samples_leaf", clf.min_samples_leaf)
      run.log("max_depth",  str(clf.max_depth))
      run.log_list("Inputs", independentcols)

      model_name = "nome_arquivo.pkl"
      filename = "outputs/" + model_name
      joblib.dump(value=clf, filename=filename)

      run.upload_file(name=model_name, path_or_stream=filename)

      run.complete()

Modelo 02 (Regressor), criado com acurácia de: [0.27472968442797563]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


Modelo 02 (Regressor), criado com acurácia de: [0.27340517254696073]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


Modelo 02 (Regressor), criado com acurácia de: [0.2752478769660701]


...

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

Modelo 02 (Regressor), criado com acurácia de: [0.7644518882939]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

Modelo 02 (Regressor), criado com acurácia de: [0.9380402449016402]


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.

- Usar o classificador acima como modelo
- Gravar log de algumas variáveis
- Gravar log da acurácia
- Exportar o modelo em arquivo
- Enviar para AZURE Cloud

In [ ]:
# Upload direto
# modelo = Model.register(workspace=workspace, model_name="ModeloUploadDireto", 
#                        model_framework=Model.Framework.SCIKITLEARN,
#                        model_framework_version='0.20.3',
#                        model_path=filename)